In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Pyspark-prepare-data").getOrCreate()
spark

In [21]:
data=[
    (1,"Anbu","laptop","electronics",1,50000,"2025-07-21"),
    (2,"Dhanu","jeans-pant","Clothing",2,1000,"2025-07-22"),
    (3,"Sippi","dog-collar","pet",2,450,"2025-07-22"),
    (4, "Alice","Smartphone","Electronics", 2,  500, "2023-01-05"),
    (5, "Bob","Laptop", "Electronics", 1, 1200, "2023-02-10"),
    (6, "Carol", "T-Shirt","Clothing", 5, 20, "2023-01-22"),
    (7, "Dave", "Jeans",  "Clothing", 2, 40, "2023-03-15"),
    (8, "Eve", "Bookshelf", "Furniture",3,150, "2023-01-12"),
    (9, "Frank","Desk", "Furniture",1,250, "2023-02-08"),
    (10, "Grace","Novel",  "Books", 4,15, "2023-01-20"),
    (11, "Heidi", "Textbook",  "Books", 2, 80, "2023-03-01"),
    (12, "Ivan","Headphones", "Electronics", 3, 100, "2023-01-30"),
    (13,"Judy","Jacket",  "Clothing", 1, 120,"2023-02-25"),
    (14,"Mallory","Coffee Table","Furniture",4,200,"2023-03-05"),
    (15,"Oscar","Notebook", "Books", 6,10, "2023-02-14")


]

df=spark.createDataFrame(data,["Orderid","name","product","category","quantity","Unitprice","orderDate"])

df.show()
df.createOrReplaceTempView("orders_local")
df.createOrReplaceGlobalTempView("orders_global")

+-------+-------+------------+-----------+--------+---------+----------+
|Orderid|   name|     product|   category|quantity|Unitprice| orderDate|
+-------+-------+------------+-----------+--------+---------+----------+
|      1|   Anbu|      laptop|electronics|       1|    50000|2025-07-21|
|      2|  Dhanu|  jeans-pant|   Clothing|       2|     1000|2025-07-22|
|      3|  Sippi|  dog-collar|        pet|       2|      450|2025-07-22|
|      4|  Alice|  Smartphone|Electronics|       2|      500|2023-01-05|
|      5|    Bob|      Laptop|Electronics|       1|     1200|2023-02-10|
|      6|  Carol|     T-Shirt|   Clothing|       5|       20|2023-01-22|
|      7|   Dave|       Jeans|   Clothing|       2|       40|2023-03-15|
|      8|    Eve|   Bookshelf|  Furniture|       3|      150|2023-01-12|
|      9|  Frank|        Desk|  Furniture|       1|      250|2023-02-08|
|     10|  Grace|       Novel|      Books|       4|       15|2023-01-20|
|     11|  Heidi|    Textbook|      Books|       2|

In [22]:
#Part A: Local View – orders_local
spark.sql("""select *
from orders_local
where category = 'Electronics' and quantity >= 2;""").show()
print("---------------------------------")

spark.sql(""" select *, quantity * unitprice as totalamount
from orders_local;""").show()
print("---------------------------------------------")
spark.sql("select category, count(*) as totalorders from orders_local group by category").show()
print("---------------------------------------------")
spark.sql("select * from orders_local where orderdate like '2023-01-%'").show()
print("---------------------------------------------")
spark.sql("select category, avg(unitprice) as avgunitprice from orders_local group by category").show()
print("---------------------------------------------")
spark.sql("select *, quantity * unitprice as totalamount from orders_local order by totalamount desc limit 1").show()
print("---------------------------------------------")



+-------+-----+----------+-----------+--------+---------+----------+
|Orderid| name|   product|   category|quantity|Unitprice| orderDate|
+-------+-----+----------+-----------+--------+---------+----------+
|      4|Alice|Smartphone|Electronics|       2|      500|2023-01-05|
|     12| Ivan|Headphones|Electronics|       3|      100|2023-01-30|
+-------+-----+----------+-----------+--------+---------+----------+

---------------------------------
+-------+-------+------------+-----------+--------+---------+----------+-----------+
|Orderid|   name|     product|   category|quantity|Unitprice| orderDate|totalamount|
+-------+-------+------------+-----------+--------+---------+----------+-----------+
|      1|   Anbu|      laptop|electronics|       1|    50000|2025-07-21|      50000|
|      2|  Dhanu|  jeans-pant|   Clothing|       2|     1000|2025-07-22|       2000|
|      3|  Sippi|  dog-collar|        pet|       2|      450|2025-07-22|        900|
|      4|  Alice|  Smartphone|Electronics

In [24]:
#partB:global view -orders
spark.sql("select *, quantity * unitprice as totalamount from global_temp.orders_global where category = 'furniture' and (quantity * unitprice) > 10000").show()
print("--------------------------------")
spark.sql("""select*,case
                         when Quantity>3 then'YES'
                         else 'NO'
                      end as DiscountFlag
              from global_temp.orders_global """).show()
print("------------------------------------")
spark.sql("select name from global_temp.orders_global group by name having count(distinct product) > 1").show()
print("---------------------------------------")
spark.sql(""" select month(orderDate),count(orderid)from global_temp.orders_global group by month(orderDate) """).show()
print("----------------------------------------")
spark.sql(""" select product,sum(quantity)as total_quantity,
rank() over( order by sum(quantity)) as rank_quantity_sold
from global_temp.orders_global
 group by product """).show()
print("--------------------")
spark.sql("select * from global_temp.orders_global").show()


+-------+----+-------+--------+--------+---------+---------+-----------+
|Orderid|name|product|category|quantity|Unitprice|orderDate|totalamount|
+-------+----+-------+--------+--------+---------+---------+-----------+
+-------+----+-------+--------+--------+---------+---------+-----------+

--------------------------------
+-------+-------+------------+-----------+--------+---------+----------+------------+
|Orderid|   name|     product|   category|quantity|Unitprice| orderDate|DiscountFlag|
+-------+-------+------------+-----------+--------+---------+----------+------------+
|      1|   Anbu|      laptop|electronics|       1|    50000|2025-07-21|          NO|
|      2|  Dhanu|  jeans-pant|   Clothing|       2|     1000|2025-07-22|          NO|
|      3|  Sippi|  dog-collar|        pet|       2|      450|2025-07-22|          NO|
|      4|  Alice|  Smartphone|Electronics|       2|      500|2023-01-05|          NO|
|      5|    Bob|      Laptop|Electronics|       1|     1200|2023-02-10|

In [28]:
# bonus challenge
spark.sql("create or replace global temp view books_global as select * from global_temp.orders_global where category = 'Books'").show()
spark.sql("select*from global_temp.books_global").show()

print("-----------------------")
spark.sql("select category, product, sumqty from (select category, product, sum(quantity) as sumqty, rank() over (partition by category order by sum(quantity) desc) as rank from global_temp.orders_global group by category, product) t where rank = 1").show()
print("-----------------------")
spark.sql("create or replace temp view filtered_orders as select * from global_temp.orders_global where category != 'clothing'").show()
spark.sql("select*from filtered_orders").show()


++
||
++
++

+-------+-----+--------+--------+--------+---------+----------+
|Orderid| name| product|category|quantity|Unitprice| orderDate|
+-------+-----+--------+--------+--------+---------+----------+
|     10|Grace|   Novel|   Books|       4|       15|2023-01-20|
|     11|Heidi|Textbook|   Books|       2|       80|2023-03-01|
|     15|Oscar|Notebook|   Books|       6|       10|2023-02-14|
+-------+-----+--------+--------+--------+---------+----------+

-----------------------
+-----------+------------+------+
|   category|     product|sumqty|
+-----------+------------+------+
|      Books|    Notebook|     6|
|   Clothing|     T-Shirt|     5|
|Electronics|  Headphones|     3|
|  Furniture|Coffee Table|     4|
|electronics|      laptop|     1|
|        pet|  dog-collar|     2|
+-----------+------------+------+

-----------------------
++
||
++
++

+-------+-------+------------+-----------+--------+---------+----------+
|Orderid|   name|     product|   category|quantity|Unitprice| o